![HPEDEVlogo](Pictures/hpedevlogo-NB.JPG)    ![Dockerlogo](Pictures/docker.png)  

Let's start by connecting to our docker appliance (Docker in Docker Appliance)

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}

In [1]:
%login 16.31.86.200

[ssh] Login to 16.31.86.200...
[ssh] host=16.31.86.200 hostname=16.31.86.200 other_conf={'user': None, 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f306417c2e0>}
[ssh] Successfully logged in.


# Using Docker
Estimated time: 15 minutes.
## The first container
In order to be able to manage a first container, the easiest approach is to import an existing one, before creating your own. For that we will refer to the public Docker registry which contains thousands of ready to be consumed containers, including this hello-world one:

In [19]:
docker run hello-world

[ssh] host = 16.31.86.200, cwd = /home/jupyter
Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
b8dfde127a29: Pulling fs layer
b8dfde127a29: Verifying Checksum
b8dfde127a29: Download complete
b8dfde127a29: Pull complete
Digest: sha256:df5f5184104426b65967e016ff2ac0bfcd44ad7899ca3bbcf8e44e4461491a9e
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container w

So we've got a success! Of course, we do not really go far first, but what can you expect from an hello-world example ;-). 
Let's understand what happened by getting informations on images and containers created with the `ls` command:

In [20]:
docker image ls

[ssh] host = 16.31.86.200, cwd = /home/jupyter
REPOSITORY      TAG       IMAGE ID       CREATED        SIZE
dockerlab1580   latest    314733fd858d   19 hours ago   1.06GB
dockerlab       latest    9149de89bb43   19 hours ago   1.06GB
test            latest    2f8bd0730896   20 hours ago   589MB
ubuntu          latest    c29284518f49   8 days ago     72.8MB
hello-world     latest    d1165f221234   4 months ago   13.3kB
centos          7         8652b9f0cb4c   8 months ago   204MB


In [22]:
docker container ls -a

[ssh] host = 16.31.86.200, cwd = /home/jupyter
CONTAINER ID   IMAGE           COMMAND                  CREATED              STATUS                          PORTS                                                                              NAMES
de0bb2409cff   hello-world     "/hello"                 About a minute ago   Exited (0) About a minute ago                                                                                      crazy_noether
4538595c00ea   dockerlab1580   "/bin/sh -c '/usr/sb…"   19 hours ago         Up 19 hours                     0.0.0.0:14005->22/tcp, :::14005->22/tcp, 0.0.0.0:14055->80/tcp, :::14055->80/tcp   dockerlab1580_dockerlab1580_1


So we see that we now have an image which has been downloaded from the Docker public registry (because it mentionned theimage wasn't found locally and was pulled), and that a container has been instantiated from that image and is not running anymore (Status Exited). The `rm` command allows to delete the container (but of course not the image which remains available)

In [24]:
docker container ls -a | grep hello-world | awk '{print $1}'| xargs docker container rm

[ssh] host = 16.31.86.200, cwd = /home/jupyter
de0bb2409cff


If everything goes well, it should have printed the Id of the container removed. Check that:

In [25]:
docker container ls -a

[ssh] host = 16.31.86.200, cwd = /home/jupyter
CONTAINER ID   IMAGE           COMMAND                  CREATED        STATUS        PORTS                                                                              NAMES
4538595c00ea   dockerlab1580   "/bin/sh -c '/usr/sb…"   19 hours ago   Up 19 hours   0.0.0.0:14005->22/tcp, :::14005->22/tcp, 0.0.0.0:14055->80/tcp, :::14055->80/tcp   dockerlab1580_dockerlab1580_1


## The second container
In order to have a more interesting environment, we'll now look for existing container images in the public Docker registry, and choose to use an ubuntu docker image on our host environment:

In [26]:
docker search ubuntu

[ssh] host = 16.31.86.200, cwd = /home/jupyter
NAME                                                      DESCRIPTION                                     STARS     OFFICIAL   AUTOMATED
ubuntu                                                    Ubuntu is a Debian-based Linux operating sys…   12533     [OK]       
dorowu/ubuntu-desktop-lxde-vnc                            Docker image to provide HTML5 VNC interface …   552                  [OK]
websphere-liberty                                         WebSphere Liberty multi-architecture images …   276       [OK]       
rastasheep/ubuntu-sshd                                    Dockerized SSH service, built on top of offi…   254                  [OK]
consol/ubuntu-xfce-vnc                                    Ubuntu container with "headless" VNC session…   241                  [OK]
ubuntu-upstart                                            Upstart is an event-based replacement for th…   113       [OK]       
1and1internet/ubuntu-16-nginx-php-ph

In [27]:
docker pull ubuntu

[ssh] host = 16.31.86.200, cwd = /home/jupyter
Using default tag: latest
latest: Pulling from library/ubuntu
Digest: sha256:b3e2e47d016c08b3396b5ebe06ab0b711c34e7f37b98c9d37abe794b71cea0a2
Status: Image is up to date for ubuntu:latest
docker.io/library/ubuntu:latest


Once the container image has been downloaded we can view it in our catalog of images:

In [28]:
docker image ls

[ssh] host = 16.31.86.200, cwd = /home/jupyter
REPOSITORY      TAG       IMAGE ID       CREATED        SIZE
dockerlab1580   latest    314733fd858d   19 hours ago   1.06GB
dockerlab       latest    9149de89bb43   19 hours ago   1.06GB
test            latest    2f8bd0730896   20 hours ago   589MB
ubuntu          latest    c29284518f49   8 days ago     72.8MB
hello-world     latest    d1165f221234   4 months ago   13.3kB
centos          7         8652b9f0cb4c   8 months ago   204MB


This content is called an image and will serve as the base to create the operational container (here based on Ubuntu) in which we will process data:

In [30]:
docker run ubuntu:latest cat /etc/debian_version

[ssh] host = 16.31.86.200, cwd = /home/jupyter
bullseye/sid


whereas on our host:

In [32]:
cat /etc/debian_version

[ssh] host = 16.31.86.200, cwd = /home/jupyter
cat: /etc/debian_version: No such file or directory


abnormal exit code: 1

This example shows that you can run completely different environments inside a container as our local Linux distribution is not based on Debian at all, contrary to Ubuntu.
Let's try to improve our container by installing wget init:

In [38]:
docker run ubuntu:latest bash -c "apt update && apt-get install -y wget"

[ssh] host = 16.31.86.200, cwd = /home/jupyter

Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [101 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [428 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [784 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [37.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [474

So you have successfully install the wgetcommand in your Ubuntu container. Hurray !
Let's try to use it:

In [39]:
docker run ubuntu:latest wget http://www.google.com

[ssh] host = 16.31.86.200, cwd = /home/jupyter
docker: Error response from daemon: OCI runtime create failed: container_linux.go:367: starting container process caused: exec: "wget": executable file not found in $PATH: unknown.


abnormal exit code: 127

Now you should start feeling depressed :-) Why isit notworking as you have seen it wasinstalled successfully just upper !!
Well, we need to understand what really a container is and the natureof the commands we passed ealier for that. Let's examine containers:

In [42]:
docker container ls -a

[ssh] host = 16.31.86.200, cwd = /home/jupyter
CONTAINER ID   IMAGE           COMMAND                  CREATED          STATUS                       PORTS                                                                              NAMES
d0f52d6a240e   ubuntu:latest   "wget http://www.goo…"   2 minutes ago    Created                                                                                                         agitated_benz
a65285a04161   ubuntu:latest   "bash -c 'apt update…"   4 minutes ago    Exited (0) 4 minutes ago                                                                                        magical_matsumoto
48f76c13b53d   ubuntu:latest   "apt-get install -y …"   5 minutes ago    Exited (100) 5 minutes ago                                                                                      hardcore_cray
f0694e523ab7   ubuntu:latest   "cat /etc/debian_ver…"   11 minutes ago   Exited (0) 11 minutes ago                                                               

You should see at least 4 Ubuntu based containers with the different commands we passed. Because each time we run `docker run`, Docker instantiates a **new** container and soesn't reuse any of the previous created. Sowe have 4 different execution environments all based on Ubuntu but all with their own lifecycle. Meaning that in the last one, the installation of `wget` never happened so the command is not found. It is in the third container only. We'll see how to deal with that later on.

Let's understand another aspect:

In [43]:
docker run ubuntu:latest uname -a

[ssh] host = 16.31.86.200, cwd = /home/jupyter
Linux 2253ec85e270 3.10.0-1160.15.2.el7.x86_64 #1 SMP Wed Feb 3 15:06:38 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


If you're on a CentOS distribution it will rather be: 
Linux ad9b474525d0 3.10.0-327.el7.x86_64 #1 SMP Thu Oct 29 17:29:29 EDT 2015 x86_64 x86_64 x86_64 GNU/Linux

So you checked that your container behaves like a Fedora 28 distribution. Only the kernel is shared between the Docker host and the Docker container. Open another console to view how this container has been created and is seen:

In [8]:
 docker run ubuntu dpkg -l

[ssh] host = 16.31.86.200, cwd = /home/jupyter
Desired=Unknown/Install/Remove/Purge/Hold
| Status=Not/Inst/Conf-files/Unpacked/halF-conf/Half-inst/trig-aWait/Trig-pend
|/ Err?=(none)/Reinst-required (Status,Err: uppercase=bad)
||/ Name                    Version                      Architecture Description
+++-=======================-============================-============-========================================================================
ii  adduser                 3.118ubuntu2                 all          add and remove users and groups
ii  apt                     2.0.6                        amd64        commandline package manager
ii  base-files              11ubuntu5.3                  amd64        Debian base system miscellaneous files
ii  base-passwd             3.5.47                       amd64        Debian base system master password and group files
ii  bash                    5.0-6ubuntu1.1               amd64        GNU Bourne Again SHell
ii  bsdutils             

For more examples and ideas, visit: http://docs.docker.com/userguide/

In [9]:
dpkg -l

[ssh] host = 16.31.86.200, cwd = /home/jupyter
/usr/bin/bash: line 1: dpkg: command not found


abnormal exit code: 127

You see that the underlying Linux distribution doesn't understand the dpkg commad. It is only valid in the container environmenet, which is an Ubuntu distribution whereas our base distribution is a CentOS one, withanother package management.

However, we can get some info on our modified Docker environment:

In [10]:
docker image ls

[ssh] host = 16.31.86.200, cwd = /home/jupyter
REPOSITORY      TAG       IMAGE ID       CREATED        SIZE
dockerlab1580   latest    314733fd858d   19 hours ago   1.06GB
dockerlab       latest    9149de89bb43   19 hours ago   1.06GB
test            latest    2f8bd0730896   19 hours ago   589MB
ubuntu          latest    c29284518f49   8 days ago     72.8MB
centos          7         8652b9f0cb4c   8 months ago   204MB


So you see that you now have 2 new images from which our 2 test containers have been created (the hello-wolrd and the Ubuntu)

In [11]:
docker container ls -a

[ssh] host = 16.31.86.200, cwd = /home/jupyter
CONTAINER ID   IMAGE           COMMAND                  CREATED         STATUS                     PORTS                                                                              NAMES
cd6fb6e8fb5b   ubuntu          "dpkg -l"                8 minutes ago   Exited (0) 8 minutes ago                                                                                      gracious_williams
ae7b579460f6   ubuntu          "dpkg -l"                9 minutes ago   Exited (0) 9 minutes ago                                                                                      elegant_ishizaka
267e71f9259f   ubuntu          "dpkg -l"                9 minutes ago   Exited (0) 9 minutes ago                                                                                      strange_faraday
f81ae8461b62   ubuntu          "dpkg-l"                 9 minutes ago   Created                                                                                         

And here you see the list of your containers with their status (they should all be Exited as they have finished their task)

In order to remove the ubuntu container(s), we first need to retrieve their container id (first column with awk) and to use docker the rm command to remove them:

In [15]:
docker container ls -a | grep ubuntu | awk '{ print $1}' | xargs docker container rm 

[ssh] host = 16.31.86.200, cwd = /home/jupyter
cd6fb6e8fb5b
ae7b579460f6
267e71f9259f
f81ae8461b62


In [ ]:
Check they are gone:

In [16]:
docker container ls

[ssh] host = 16.31.86.200, cwd = /home/jupyter
CONTAINER ID   IMAGE           COMMAND                  CREATED        STATUS        PORTS                                                                              NAMES
4538595c00ea   dockerlab1580   "/bin/sh -c '/usr/sb…"   19 hours ago   Up 19 hours   0.0.0.0:14005->22/tcp, :::14005->22/tcp, 0.0.0.0:14055->80/tcp, :::14055->80/tcp   dockerlab1580_dockerlab1580_1


Let's list the images again!

In [ ]:
docker search fedora

In [ ]:
docker pull fedora

Once the container image has been downloaded we can view it in our catalog of images:

In [ ]:
docker images

This content is called an image and will serve as the base to create the operational container (here based on Fedora) in which we will process data:

recup id here + comment

In [ ]:
docker run -ti cc510acfcd70 /bin/bash

In [ ]:
cat /etc/fedora-release

In [ ]:
dnf install -y wget

In [ ]:
uname -a

If you're on a CentOS distribution it will rather be: 
Linux ad9b474525d0 3.10.0-327.el7.x86_64 #1 SMP Thu Oct 29 17:29:29 EDT 2015 x86_64 x86_64 x86_64 GNU/Linux

So you checked that your container behaves like a Fedora 28 distribution. Only the kernel is shared between the Docker host and the Docker container. Open another console to view how this container has been created and is seen:

In [ ]:
docker ps

If you logout of the container, you'll see how Docker manages that fact:

In [ ]:
exit

In [ ]:
docker ps

In [ ]:
docker ps -a

So your container is seen as stopped and you have to use the -a option to see it in the history of containers created, but not active anymore.

Re-create a new container based on the same image, connect to it and look at the packages installed. Check what Docker sees. Use the previous commands to perform these tasks.

Answer the questions:
  1. Can you download a web page with wget in your container ? Why ? Which steps are needed now ? Why ?
  2. Can you connect back to your first container ? (Hint: use **docker start** to re-enable your dead container and **docker attach** to re-enter in it)
  3. Feel free to call the trainer if something is unclear or if you want to ensure you understand all points.


# Configuring owncloud in a container

Estimated time: 60 minutes.

Based on the work done in the Docker Dojo during a Grenoble Docker Meetup (cf: https://github.com/Enalean/docker-dojo/tree/master/owncloud).

Owncloud is a web based application providing services such as calendar data or file sharing e.g.
When we want to contain an application such as owncloud, there are a certain number of aspects to take in account and solve:
  1. installing the application and its dependencies in the container
  2. allow IP configuration for remote access to the application
  3. allow data persistence at each invocation of the container
  4. allow configuration data persistence at each invocation of the container
One possibility would be to run the container from an image and launch the various commands in the container (as we've done previously). We could put that in a script and launch it systematically when we instantiate a container from an image, or rebuild a prepared image to be instantiated later. But there is a better way to achieve what we want to do, and this is by using the automation process by Docker with the Dockerfile.

The Dockerfile is a way to describe all the operations required to create an image from an initial empty one and stacking all the operations to build at the end the final image ready to be instantiated and consumed and thrown away
Let's start our Dockerfile by creating a simple container from a base image and just installing some software components useful for our environment, and build an image from that:


`#` **`cat > Dockerfile << EOF`**
```
FROM centos:6
RUN yum install -y httpd
EOF
```

Note : Use **`cat Dockerfile`** to see the file content.

In [ ]:
docker build .

In [ ]:
docker images

In [ ]:
So we can verify that a new CentOS 6 image has been downloaded and based on it a new image has been created (without name nor tag, just an ID) containing httpd installed with its dependencies. Check it by instantiating a container based on that image and launching httpd in it:

`#` **`**

`[root@babbfd33b239 /]#` **` `**
```


`[root@babbfd33b239 /]#` **`t`**

In [ ]:
docker run -ti 358657a2b6b0 /bin/bash

In [ ]:
httpd

httpd: Could not reliably determine the server's fully qualified domain name, using 172.17.0.5 for ServerName
```
Note : The above message is just a warning, as an evidence, you can see the processes running with the next command.

In [ ]:
ps auxww |grep ht

In [ ]:
exit

In [ ]:
docker ps -a

In [ ]:
docker diff babbfd33b239

In [ ]:
docker history 358657a2b6b

So we checked that we can launch the httpd server from inside an instantiated container made from our image. We also checked how our image was built. Note that the image built is 72 MB larger than the base CentOS 6 one (shown by history) and has sensible modifications shown by the diff command.
It's a good start, but now we would like to have our httpd server started automatically with our container creation. And have attribution accordingly ;-)

In [ ]:
cat >> Dockerfile

In [ ]:
```
MAINTAINER myself@mydomain.org
CMD httpd
EOF
```

In [ ]:
docker build .

You can remark that all the first steps are very quick. This is because Docker caches steps, and will not repeat them unless the Dockerfile changes. You can modify the Docker file by putting the `MAINTAINER` command as the second line and re-launch the build. You'll see that in that case Docker invalidates its cache and restarts.
Now start a container from that image to check the web server is indeed started

In [ ]:
docker run -ti 76cec1da7808

httpd: Could not reliably determine the server's fully qualified domain name, using 172.17.0.6 for ServerName
```
1. What happened ? Why don't you get a prompt ?
2. Use `docker ps` to see the status of your container and `docker logs` to see what happened.
3. Try to adapt the Dockerfile to solve that issue. **Discuss with your trainer if you're stuck !**

If the **`-ti`** option makes Apache die, then use the **`-d`** option instead to launch the container as a daemon instead.

In [ ]:
perl -pi -e 's|D httpd|D /usr/sbin/apachectl -DFOREGROUND -k start|' Dockerfile

(This magic command replaces the launch of the httpd command by the apachectl one with the right options. If you are using CentOS 7 as the host OS, you will have to install perl via yum).

1. Try to use a browser (you may want to install lynx) to connect to your web server. Can you do it ?
2. Which IP address do you point to ? You can use `docker exec` to get the IP address for the container.

By default, the container ports are not exposed outside of the container. So you can't use your host OS to access your isolated webserver. If you are running the container locally navigate to http://localhost in the host browser. If the container is running on lab instruture navigate to http://10.3.222.x (as directed by the instructor).

You will have to explicitly open container port 80 to allow access to the web server running in the container. This will require changes to the Dockerfile:

In [ ]:
cat >> Dockerfile << EOF`

In [ ]:
EXPOSE 80
EOF

Note : This is not mandatory, as this is only metadata for the image. However it will help people to know the exposed port of your image.

In [ ]:
docker build .

In [ ]:
docker run -d -p 80:80 04d9c18da22a
docker ps

Now that we have exposed the port, we're able to launch our container in daemon mode (-d) and redirect the local port 80 to the container port 80. Our web server is listening is listening on port 80 in the container. Repeat the earlier attempt to connect to the web server in the container. You should see a CentOS based page on your host distribution.

It's now time to add some content to our web server !
Modify again the Dockerfile to add owncloud to our image:

In [ ]:
cat >> Dockerfile << EOF

In [ ]:
RUN yum install -y tar bzip2
ADD https://download.owncloud.org/community/7.0/owncloud-7.0.15.tar.bz2 /var/www/html/
# Add this only if before docker engine 17.03
# RUN cd /var/www/html/ && tar xvfj owncloud-7.0.15.tar.bz2 && rm -f owncloud-7.0.15.tar.bz2
EOF

We can directly point to a URL, Docker will download the content and extract it in place.
Try now to connect to your owncloud instance. The URL is http://10.3.222.X/owncloud if we host the lab or http://localhost/owncloud if you run it locally.

![Owncloud failed](Pictures/owncloud_without_dep.png)

1. What happens?
2. What should you do next to solve the issue ? **Discuss with your trainer if you're stuck !**

Hint, you probably need to add the owncloud dependencies to be able to launch it. Open your Dockerfile and add the following line after the last ADD

In [ ]:
RUN yum install -y php php-dom php-mbstring php-pdo php-gd

In [ ]:
With that you should be able to use owncloud ! (Note that you need to use that version with CentOS 6 for a PHP dependency management) But we're not done yet !!!
If you log on to your owncloud instance, and start customizing it (login/passwd for admin, storage path), you'll have errors first, that we'll fix later on and then if you `Docker stop` and `Docker rm` the container to relaunch it again, of course, none of this customization will be kept as it's not part of your container content.

So we now have to deal with storage management for our Docker container. First we need to solve the error generated when you tried to configure your owncloud instance. We had rights issues. Use the following command to help solve the issue:

In [ ]:
docker exec b42f9f6f1034 ls -al /var/www/html
docker exec b42f9f6f1034 ps auxww | grep httpd

The principle is that the owner of the httpd process should have the rights on the owncloud directory to read and store files there. ** So modify you Dockerfile accordingly and retest **.

Now you should be able to customize your owncloud instance and start using it.

By now you have probably remarked that the ADD order is done each time, without any benefit from the cache management of Docker. Also you have to each time deal with IDs for containers and images, which is not that convenient. Let's fix that. Download the owncloud tar file in your directory and modify the ADD line:

In [ ]:
wget https://download.owncloud.org/community/7.0/owncloud-7.0.15.tar.bz2

In [ ]:
perl -pi -e 's|ADD https://download.owncloud.org/community/7.0/owncloud-7.0.15.tar.bz2|COPY owncloud-7.0.15.tar.bz2|' Dockerfile

In [ ]:
docker build -t owncloud .

Next time you re-run the build, the cache effect is optimal. Also you now have tagged your image and use it by its name:

docker images

It would be great if you could persist the content from one run to another.  Yes, you can ;-) For that, you need to attach a local directory of your host to your container, and point the setup of your owncloud to that directory instead of the one under `/var/www/html/owncloud`.
Create a `/data` directory on your host, mount it in your container under `/data`, and then point your setup ot it:

In [ ]:
mkdir -p /data

In [ ]:
date > /data/myfile.txt

In [ ]:
cat >> Dockerfile << EOF

In [ ]:
```
VOLUME /data
EOF
```

In [ ]:
docker build -t owncloud .

In [ ]:
docker ps

In [ ]:
docker stop 29c8f5ca3d76 

In [ ]:
docker rm 29c8f5ca3d76

In [ ]:
docker run -d -p 80:80 -v /data:/data owncloud:latest

Now reload the owncloud configuration page in your browser, but this time configure the data folder as in the following screen shot:

![Owncloud Setup](Pictures/owncloud.png)

** If you encounter issues you need to adapt your environment so that the apache user is allowed to write on to the /data directory. **

Your current Dockerfile should look like this at that point:

In [ ]:
cat Dockerfile

In [ ]:
```
FROM centos:6
#FROM fedora:latest
RUN yum install -y httpd
MAINTAINER myself@mydomain.org
RUN yum install -y tar bzip2
COPY owncloud-7.0.15.tar.bz2 /var/www/html/
RUN cd /var/www/html/ && tar xvfj owncloud-7.0.15.tar.bz2 && rm -f owncloud-7.0.15.tar.bz2
RUN yum install -y php php-dom php-mbstring php-pdo php-gd
VOLUME /data
RUN chown -R apache:apache /var/www/html/owncloud /data
CMD /usr/sbin/apachectl -DFOREGROUND -k start
EXPOSE 80
```

Move the example text file you created earlier to your ownClould Documents folder so you can see the file and view the file in ownCloud.

In [ ]:
mv /data/myfile.txt /data/bruno/files/Documents

Open the Documents folder in the ownCloud Web UI. Confirm that the myfile.txt example file is present and then view the contents to check that they match what you created earlier.


In [ ]:
docker ps

In [ ]:
docker stop 23f
docker rm 23f

In [ ]:
docker run -d -p 80:80 -v /data:/data owncloud:latest

1. At that point you should find again your data on your owncloud instance right ? But what additional pain point do you have ?
2. Knowing that the owncloud configuration data are located under `/var/www/html/owncloud/config/config.php`  try to adapt the Dockerfile to solve that last issue. **Discuss with your trainer if you're stuck !**
Note : there is more than one way to solve this.

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#551199;"></i>&nbsp;&nbsp;Next Steps

# Lab3 : Using Docker Compose

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="3-WKSHP-Using-Docker-Compose.ipynb" target="New" title="Next LAB: Using Docker Compose"><i class="fas fa-chevron-circle-right" style="color:#551199;"></i></a></h2>

</br>
 <a href="1-WKSHP-Intro-to-Containers-techno.ipynb" target="New" title="Back: Introduction to Containers technologies"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="3-WKSHP-Using-Docker-Compose.ipynb" target="New" title="Next:Using Docker Compose"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
